In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns

import harness

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)

In [4]:
add_features = harness.chain(
    harness.add_date_parts, harness.add_haul, harness.add_weather
)

In [5]:
flights_with_features = add_features(flights_clean)

In [6]:
x_transform = harness.chain(
    harness.keep_only_test_columns,
    add_features,
    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric,
    harness.scale
)

In [7]:
transformer = harness.DataTransformer(
    x_transform
)

In [8]:
x_train, y_train = transformer.extract_transform(flights_clean)

In [14]:
def train_model():
    linreg = lm.Ridge(random_state=42)
    grid_search = ms.GridSearchCV(
        linreg, dict(alpha=[10 ** i for i in range(-6, 7)])
    )
    grid_search.fit(x_train.values, y_train.values)
    print(grid_search.best_score_)
    harness.save(
        grid_search.best_estimator_,
        'everything_linear_no_transform_model'
    )

In [15]:
train_model()

0.09289811448296384


In [11]:
linreg = harness.load('everything_linear_no_transform_model')
linreg_model = harness.TrainedModel(linreg, transformer)

In [12]:
linreg_model.validate(harness.clean_train(flights_test))

-0.03860541226920455

In [13]:
# linreg_model.submit(
#     'data/test.csv', 'everything_linear_no_transform_submission.csv', 'predicted_delay'
# )